## Imports

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost
import optuna
import time
import lightgbm as lgb
import os
from sklearn.metrics import root_mean_squared_error
from scipy.optimize import minimize as sp_minimize
from sklearn.preprocessing import StandardScaler
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

## Configs

In [51]:
train_path = '../data/train.csv'
test_path = '../data/test_for_participants.csv'
sample_path = '../data/sample_submission.csv'

# ── Dual Validation Date Boundaries ──────────────────────────────
# Val Set 1 (Physics): Autumn/Winter 2024 — cold, dark, windy grid
VAL_PHYSICS_START = '2024-09-01'
VAL_PHYSICS_END   = '2025-01-01'
# Val Set 2 (Recency): Summer 2025 — last 3 months before LB
VAL_RECENCY_START = '2025-06-01'

SEED = 42
N_TRIALS_LGB = 30
SAVED_LGB_PATH = '../models/lgb_final.txt'
ROUND_MULTIPLIER = 1.15

os.makedirs('../models', exist_ok=True)

# Set random seed for reproducibility
np.random.seed(SEED)

## Data loading

In [52]:
train_raw = pd.read_csv(train_path)
test_raw = pd.read_csv(test_path)
sample_sub = pd.read_csv(sample_path)

In [53]:
for df in [train_raw, test_raw]:
    df['delivery_start'] = pd.to_datetime(df['delivery_start'])
    df['delivery_end'] = pd.to_datetime(df['delivery_end'])

train_raw['is_test'] = 0
test_raw['is_test'] = 1
test_raw['target'] = np.nan

df = pd.concat([train_raw, test_raw], ignore_index=True)
df = df.sort_values(['market', 'delivery_start']).reset_index(drop=True)
df.head()

,id,target,market,global_horizontal_irradiance,diffuse_horizontal_irradiance,direct_normal_irradiance,cloud_cover_total,cloud_cover_low,cloud_cover_mid,cloud_cover_high,...,wind_speed_80m,wind_direction_80m,wind_gust_speed_10m,wind_speed_10m,solar_forecast,wind_forecast,load_forecast,delivery_start,delivery_end,is_test
0,0,-1.913,Market A,0.0,0.0,0.0,2.0,0.0,0.0,2.0,...,31.253719,245.501450,25.199999,15.077082,0.0,24050.1,38163.0100,2023-01-01 00:00:00,2023-01-01 01:00:00,0
1,5,-0.839,Market A,0.0,0.0,0.0,15.0,0.0,0.0,15.0,...,30.918108,242.241547,23.400000,14.186923,0.0,23886.3,37379.1898,2023-01-01 01:00:00,2023-01-01 02:00:00,0
2,10,-1.107,Market A,0.0,0.0,0.0,17.0,0.0,0.0,17.0,...,26.983196,224.999893,21.240000,12.413477,0.0,23366.5,36336.8303,2023-01-01 02:00:00,2023-01-01 03:00:00,0
3,15,0.035,Market A,0.0,0.0,0.0,16.0,0.0,0.0,16.0,...,22.218153,229.600174,16.199999,10.483357,0.0,22829.8,35337.7595,2023-01-01 03:00:00,2023-01-01 04:00:00,0
4,20,-0.829,Market A,0.0,0.0,0.0,10.0,0.0,0.0,10.0,...,27.210381,244.113022,18.359999,11.918120,0.0,22347.6,34474.3403,2023-01-01 04:00:00,2023-01-01 05:00:00,0


In [63]:
df.target.min()

np.float64(-251.794)

## Enhanced Feature Engineering - Ultra Advanced Version

In [54]:
# Basic time features
ds = df["delivery_start"]
df["hour"]         = ds.dt.hour
df["day_of_week"]  = ds.dt.dayofweek
df["day_of_month"] = ds.dt.day
df["month"]        = ds.dt.month
df["quarter"]      = ds.dt.quarter
df["day_of_year"]  = ds.dt.dayofyear
df["year"]         = ds.dt.year
df["is_weekend"]   = (ds.dt.dayofweek >= 5).astype(np.int8)
df["week_of_year"] = ds.dt.isocalendar().week.astype(int)

# Cyclical encoding for time features
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['day_of_year_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
df['day_of_year_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)

# Market encoding
market_map = {f"Market {c}": i for i, c in enumerate("ABCDEF")}
df["market_id"] = df["market"].map(market_map).astype(np.int8)

# Advanced demand and supply features
df["residual_demand"] = df["load_forecast"] - df["solar_forecast"] - df["wind_forecast"]
df["supply_ratio"] = (df["solar_forecast"] + df["wind_forecast"]) / (df["load_forecast"] + 1)
df["renewable_ratio"] = (df["solar_forecast"] + df["wind_forecast"]) / (df["solar_forecast"] + df["wind_forecast"] + df["load_forecast"] + 1)
df["net_supply"] = df["solar_forecast"] + df["wind_forecast"]
df["demand_supply_balance"] = df["load_forecast"] / (df["solar_forecast"] + df["wind_forecast"] + 1)

# Tightness ratios
df["tightness_ratio"] = df["residual_demand"] / (df["load_forecast"] + 1)
df["tightness_x_month"] = df["tightness_ratio"] * df["month"]
df["tightness_x_hour"] = df["tightness_ratio"] * df["hour"]
df["tightness_x_dow"] = df["tightness_ratio"] * df["day_of_week"]

# Price sensitivity indicators
df["solar_wind_ratio"] = df["solar_forecast"] / (df["wind_forecast"] + 1)
df["wind_solar_ratio"] = df["wind_forecast"] / (df["solar_forecast"] + 1)

In [55]:
# ── Advanced Weather Physics Features ──────────────────────────────────────
# First, alias columns whose names differ between dataset and our code

if 'convective_available_potential_energy' in df.columns:
    df['cape'] = df['convective_available_potential_energy']
if 'precipitation_amount' in df.columns:
    df['precipitation'] = df['precipitation_amount']
if 'apparent_temperature_2m' in df.columns:
    df['apparent_temperature'] = df['apparent_temperature_2m']
if 'freezing_level_height' in df.columns:
    df['boundary_layer_height'] = df['freezing_level_height']

# Estimate missing columns from available physics
# Saturation vapour pressure (Tetens formula)
es = 6.112 * np.exp((17.67 * df['air_temperature_2m']) / (df['air_temperature_2m'] + 243.5))
ea = (df['relative_humidity_2m'] / 100.0) * es
df['vapour_pressure_deficit_2m'] = es - ea

# Proxy precipitation probability from relative humidity
df['precipitation_probability'] = np.where(df['relative_humidity_2m'] > 85, 50, 0)

# ── Derived weather features ──
df['dew_point_depression']  = df['air_temperature_2m'] - df['dew_point_temperature_2m']
df['wet_bulb_depression']   = df['air_temperature_2m'] - df['wet_bulb_temperature_2m']
df['humidity_ratio']        = (0.622 * df['vapour_pressure_deficit_2m']) / (df['surface_pressure'] - df['vapour_pressure_deficit_2m'])
df['blh_normalized_pressure'] = df['boundary_layer_height'] / (df['surface_pressure'] / 1000)

# Wind shear (10m vs 80m)
df['wind_shear']       = df['wind_speed_80m'] - df['wind_speed_10m']
df['wind_shear_ratio'] = df['wind_speed_80m'] / (df['wind_speed_10m'] + 0.1)

# Convection indices
df['cape_cin_interaction'] = df['cape'] * df['convective_inhibition']
df['convection_potential'] = df['cape'] / (abs(df['convective_inhibition']) + 1)

# Visibility & cloud
df['visibility_cloud_interaction'] = df['visibility'] / (df['cloud_cover_total'] + 1)

# Combined weather severity index
df['weather_severity'] = (
    df['cloud_cover_total'] / 100 +
    (100 - df['visibility'].clip(0, 100)) / 100 +
    df['precipitation_probability'] / 100 +
    df['cape'] / 1000
) / 4

# Solar / wind potential
df['solar_potential'] = df['global_horizontal_irradiance'] * (1 - df['cloud_cover_total'] / 100)
df['wind_potential']  = df['wind_speed_80m'] ** 3  # cubic ∝ power

# Extreme weather flags
df['extreme_temp']   = ((df['air_temperature_2m'] > 30) | (df['air_temperature_2m'] < -5)).astype(int)
df['extreme_wind']   = (df['wind_speed_80m'] > 25).astype(int)
df['extreme_precip'] = (df['precipitation'] > 5).astype(int)

# Seasonal × weather interactions
df['temp_month_interaction'] = df['air_temperature_2m'] * df['month']
df['wind_month_interaction'] = df['wind_speed_80m'] * df['month']
df['solar_hour_interaction'] = df['solar_forecast'] * df['hour']

# Heating / cooling degree-hours
df['cooling_degree_hours'] = np.maximum(df['air_temperature_2m'] - 22, 0)
df['heating_degree_hours'] = np.maximum(18 - df['air_temperature_2m'], 0)

# Vapour pressure deficit normalised
df['vpd_normalized'] = df['vapour_pressure_deficit_2m'] / df['surface_pressure']

# Apparent temperature anomaly
df['apparent_temp_anomaly']   = df['apparent_temperature'] - df['air_temperature_2m']
df['apparent_air_temp_ratio'] = df['apparent_temperature'] / (df['air_temperature_2m'] + 1)

# Lifted Index feature (strong storm predictor)
if 'lifted_index' in df.columns:
    df['lifted_index_negative'] = (-df['lifted_index']).clip(lower=0)  # only instability

In [56]:
# ── Weather Momentum & Lag Features (NO target leakage) ────────────────────
weather_cols = ['wind_speed_80m', 'solar_forecast', 'load_forecast',
                'wind_forecast', 'air_temperature_2m']

for col in weather_cols:
    grp = df.groupby('market_id')[col]

    # Hourly differences (momentum)
    df[f'{col}_diff_1h']  = grp.diff(1)
    df[f'{col}_diff_3h']  = grp.diff(3)
    df[f'{col}_diff_6h']  = grp.diff(6)
    df[f'{col}_diff_12h'] = grp.diff(12)

    # Rolling mean / std
    df[f'{col}_rolling_mean_6h']  = grp.transform(lambda x: x.rolling(6,  min_periods=1).mean())
    df[f'{col}_rolling_std_6h']   = grp.transform(lambda x: x.rolling(6,  min_periods=1).std().fillna(0))
    df[f'{col}_rolling_mean_24h'] = grp.transform(lambda x: x.rolling(24, min_periods=1).mean())
    df[f'{col}_rolling_std_24h']  = grp.transform(lambda x: x.rolling(24, min_periods=1).std().fillna(0))

    # Rolling min / max (use bfill() instead of deprecated fillna(method=...))
    df[f'{col}_rolling_min_24h'] = grp.transform(lambda x: x.rolling(24, min_periods=1).min().bfill())
    df[f'{col}_rolling_max_24h'] = grp.transform(lambda x: x.rolling(24, min_periods=1).max().bfill())
    df[f'{col}_range_24h']       = df[f'{col}_rolling_max_24h'] - df[f'{col}_rolling_min_24h']

    # Exponential weighted moving averages
    df[f'{col}_ewm_6h']  = grp.transform(lambda x: x.ewm(span=6,  adjust=False).mean())
    df[f'{col}_ewm_24h'] = grp.transform(lambda x: x.ewm(span=24, adjust=False).mean())

    # Z-score vs rolling window
    df[f'{col}_zscore_24h'] = (df[col] - df[f'{col}_rolling_mean_24h']) / (df[f'{col}_rolling_std_24h'] + 0.001)

# Temperature anomaly vs recent history
df['temp_24h_mean']    = df.groupby('market_id')['air_temperature_2m'].transform(lambda x: x.rolling(24, min_periods=1).mean())
df['temp_72h_mean']    = df.groupby('market_id')['air_temperature_2m'].transform(lambda x: x.rolling(72, min_periods=1).mean())
df['temp_anomaly_24h'] = df['air_temperature_2m'] - df['temp_24h_mean']
df['temp_anomaly_72h'] = df['air_temperature_2m'] - df['temp_72h_mean']

# Wind direction components & stability
df['wind_dir_sin']    = np.sin(np.deg2rad(df['wind_direction_80m']))
df['wind_dir_cos']    = np.cos(np.deg2rad(df['wind_direction_80m']))
df['wind_dir_change'] = df.groupby('market_id')['wind_direction_80m'].diff(1).abs()

# Pressure & humidity interactions with temperature
df['pressure_temp_interaction']  = df['surface_pressure'] * df['air_temperature_2m']
df['humidity_temp_interaction']  = df['relative_humidity_2m'] * df['air_temperature_2m']
df['pressure_gradient']          = df.groupby('market_id')['surface_pressure'].diff(1)

# Cloud & precipitation transforms
df['cloud_cover_total_sq']  = df['cloud_cover_total'] ** 2
df['cloud_cover_effect']    = df['cloud_cover_total'] * df['global_horizontal_irradiance']
df['precip_prob_sq']        = df['precipitation_probability'] ** 2
df['precip_effect']         = df['precipitation'] * df['precipitation_probability']

# Radiation efficiency
df['solar_efficiency']       = df['solar_forecast'] / (df['global_horizontal_irradiance'] + 1)
df['radiation_cloud_ratio']  = df['global_horizontal_irradiance'] / (df['cloud_cover_total'] + 1)

In [57]:
# Advanced temporal features

# Hourly and daily patterns
df['hour_from_peak'] = abs(df['hour'] - 12)  # Distance from peak solar hour
df['is_peak_solar'] = ((df['hour'] >= 10) & (df['hour'] <= 16)).astype(int)
df['is_off_peak'] = ((df['hour'] >= 22) | (df['hour'] <= 6)).astype(int)
df['is_business_hours'] = ((df['hour'] >= 8) & (df['hour'] <= 18) & (df['day_of_week'] < 5)).astype(int)

# Week patterns
df['is_monday'] = (df['day_of_week'] == 0).astype(int)
df['is_friday'] = (df['day_of_week'] == 4).astype(int)
df['is_weekend_start'] = (df['day_of_week'] == 4).astype(int)  # Friday
df['is_weekend_end'] = (df['day_of_week'] == 6).astype(int)    # Sunday

# Monthly patterns
df['is_winter'] = df['month'].isin([12, 1, 2]).astype(int)
df['is_summer'] = df['month'].isin([6, 7, 8]).astype(int)
df['is_spring'] = df['month'].isin([3, 4, 5]).astype(int)
df['is_autumn'] = df['month'].isin([9, 10, 11]).astype(int)

# Quarter interactions
df['q1_temp_interaction'] = (df['quarter'] == 1) * df['air_temperature_2m']
df['q2_temp_interaction'] = (df['quarter'] == 2) * df['air_temperature_2m']
df['q3_temp_interaction'] = (df['quarter'] == 3) * df['air_temperature_2m']
df['q4_temp_interaction'] = (df['quarter'] == 4) * df['air_temperature_2m']

# Seasonal demand patterns
df['winter_load_factor'] = df['is_winter'] * df['load_forecast']
df['summer_load_factor'] = df['is_summer'] * df['load_forecast']
df['spring_load_factor'] = df['is_spring'] * df['load_forecast']
df['autumn_load_factor'] = df['is_autumn'] * df['load_forecast']

In [58]:
# ══════════════════════════════════════════════════════════════════════════════
# Historical Target Encoding  (VALIDATION-SAFE MODE — v6 Dual Val)
# ══════════════════════════════════════════════════════════════════════════════
# For tuning/validation: compute group means ONLY from TRAINING data
# (excluding both validation sets) to avoid any leakage.

# ⚠️ VALIDATION MODE: using only training data (pre-Physics val start)
# For final submission, change to:
#     strict_train = df[(df['is_test'] == 0) & (df['delivery_start'] < VAL_PHYSICS_START)]
strict_train = df[(df['is_test'] == 0) & (df['delivery_start'] < VAL_PHYSICS_START)]

# ── Market × Hour mean ──
mean_mh = (strict_train
    .groupby(['market_id', 'hour'])['target']
    .mean().reset_index(name='target_histmean_mh'))

# ── Market × Day-of-week mean ──
mean_mdow = (strict_train
    .groupby(['market_id', 'day_of_week'])['target']
    .mean().reset_index(name='target_histmean_mdow'))

# ── Market × Month mean ──
mean_mm = (strict_train
    .groupby(['market_id', 'month'])['target']
    .mean().reset_index(name='target_histmean_mm'))

# ── Market mean (global baseline per market) ──
mean_m = (strict_train
    .groupby(['market_id'])['target']
    .mean().reset_index(name='target_histmean_m'))

# ── Hour mean (global baseline per hour) ──
mean_h = (strict_train
    .groupby(['hour'])['target']
    .mean().reset_index(name='target_histmean_h'))

# ── Market × Hour × DayOfWeek mean ──
mean_mhd = (strict_train
    .groupby(['market_id', 'hour', 'day_of_week'])['target']
    .mean().reset_index(name='target_histmean_mhd'))

# ── Market × Quarter mean ──
mean_mq = (strict_train
    .groupby(['market_id', 'quarter'])['target']
    .mean().reset_index(name='target_histmean_mq'))

# Merge ALL onto full dataframe
df = df.merge(mean_mh,  on=['market_id', 'hour'],          how='left')
df = df.merge(mean_mdow, on=['market_id', 'day_of_week'],  how='left')
df = df.merge(mean_mm,  on=['market_id', 'month'],          how='left')
df = df.merge(mean_m,   on=['market_id'],                   how='left')
df = df.merge(mean_h,   on=['hour'],                        how='left')
df = df.merge(mean_mhd, on=['market_id', 'hour', 'day_of_week'], how='left')
df = df.merge(mean_mq,  on=['market_id', 'quarter'],       how='left')

# Fill any NaN hist-means with global training mean
global_mean = strict_train['target'].mean()
for c in [c for c in df.columns if c.startswith('target_histmean_')]:
    df[c] = df[c].fillna(global_mean)

# ── Deviations from historical baselines (computed from safe features only) ──
df['histmean_mh_x_residual'] = df['target_histmean_mh'] * df['residual_demand']
df['histmean_mh_x_tightness'] = df['target_histmean_mh'] * df['tightness_ratio']
df['histmean_deviation_dow_vs_m'] = df['target_histmean_mdow'] - df['target_histmean_m']
df['histmean_deviation_mh_vs_h'] = df['target_histmean_mh'] - df['target_histmean_h']

print(f"✅ Safe historical encoding: {sum(c.startswith('target_histmean') for c in df.columns)} features")
print(f"   Global training mean: {global_mean:.4f}")

✅ Safe historical encoding: 7 features
   Global training mean: 37.3241


In [59]:
# ── Interaction features (weather × demand, cross-market) ──────────────────
df['temp_load_interaction']  = df['air_temperature_2m'] * df['load_forecast']
df['wind_load_interaction']  = df['wind_speed_80m'] * df['load_forecast']
df['solar_load_interaction'] = df['solar_forecast'] * df['load_forecast']
df['temp_wind_interaction']  = df['air_temperature_2m'] * df['wind_speed_80m']
df['temp_solar_interaction'] = df['air_temperature_2m'] * df['solar_forecast']
df['wind_solar_interaction'] = df['wind_speed_80m'] * df['solar_forecast']

# Triple interactions
df['temp_wind_load_interaction']  = df['air_temperature_2m'] * df['wind_speed_80m'] * df['load_forecast']
df['temp_solar_load_interaction'] = df['air_temperature_2m'] * df['solar_forecast'] * df['load_forecast']
df['wind_solar_load_interaction'] = df['wind_speed_80m'] * df['solar_forecast'] * df['load_forecast']

# Weather volatility (rolling on weather — safe, no target)
df['temp_volatility']  = df.groupby('market_id')['air_temperature_2m'].transform(lambda x: x.rolling(24, min_periods=1).std().fillna(0))
df['wind_volatility']  = df.groupby('market_id')['wind_speed_80m'].transform(lambda x: x.rolling(24, min_periods=1).std().fillna(0))
df['solar_volatility'] = df.groupby('market_id')['solar_forecast'].transform(lambda x: x.rolling(24, min_periods=1).std().fillna(0))

# Rate of change (weather — safe)
df['temp_rate_change']  = df.groupby('market_id')['air_temperature_2m'].diff(1) / (df.groupby('market_id')['air_temperature_2m'].shift(1).abs() + 0.01)
df['wind_rate_change']  = df.groupby('market_id')['wind_speed_80m'].diff(1) / (df.groupby('market_id')['wind_speed_80m'].shift(1).abs() + 0.01)
df['solar_rate_change'] = df.groupby('market_id')['solar_forecast'].diff(1) / (df.groupby('market_id')['solar_forecast'].shift(1).abs() + 0.01)

# Cross-market features (same timestamp, across markets — safe)
for col in ['wind_speed_80m', 'solar_forecast', 'load_forecast']:
    ts_mean = df.groupby('delivery_start')[col].transform('mean')
    ts_std  = df.groupby('delivery_start')[col].transform('std') + 0.001
    df[f'{col}_market_diff']   = df[col] - ts_mean
    df[f'{col}_market_zscore'] = (df[col] - ts_mean) / ts_std

# Advanced rolling statistics (weather — safe)
for col in ['wind_speed_80m', 'solar_forecast', 'load_forecast']:
    df[f'{col}_skew_24h'] = df.groupby('market_id')[col].transform(
        lambda x: x.rolling(24, min_periods=12).skew().fillna(0))


# ── SUMMER HEATWAVE & GRID STRESS ──────────────────────────────────────────────
# 1. Heatwave Penalty (Exponential stress when Temp > 25C and Wind is low)
df['heatwave_stress'] = np.where(
    (df['air_temperature_2m'] > 25) & (df['wind_speed_80m'] < 5),
    (df['air_temperature_2m'] - 25) ** 2, 
    0
)

# 2. Solar/Wind Drought Flag
df['renewable_drought'] = ((df['solar_forecast'] < 10) & (df['wind_forecast'] < 10)).astype(int)

# 3. Summer Cooling Load Proxy
df['cooling_degree_load'] = df['load_forecast'] * np.maximum(0, df['air_temperature_2m'] - 22)

# ── Final NaN handling ─────────────────────────────────────────────────────
# NEVER fill target NaN — those are test rows!
exclude_from_fill = {'target', 'delivery_start', 'delivery_end', 'market', 'id'}
for col in df.columns:
    if col in exclude_from_fill:
        continue
    if df[col].dtype in ['float64', 'float32', 'int64', 'int32', 'int8']:
        nan_count = df[col].isna().sum()
        if nan_count > 0:
            df[col] = df[col].fillna(df[col].median())

print(f"✅ Feature engineering complete: {len(df.columns)} total columns")
print(f"   Training rows: {(df['is_test']==0).sum()}, Test rows: {(df['is_test']==1).sum()}")
print(f"   NaN check (target): {df['target'].isna().sum()} NaN (should equal test rows)")

✅ Feature engineering complete: 235 total columns
   Training rows: 132608, Test rows: 13098
   NaN check (target): 13098 NaN (should equal test rows)


## Prepare X and y

In [41]:
observed_df = df[df['is_test'] == 0].copy()
test_df = df[df['is_test'] == 1].copy()

# ══════════════════════════════════════════════════════════════════
# DUAL VALIDATION SPLITTING LOGIC
# ══════════════════════════════════════════════════════════════════
# 1. Val Set 1: The "Physics" Test (Autumn/Winter 2024)
mask_val_physics = (observed_df['delivery_start'] >= VAL_PHYSICS_START) & (observed_df['delivery_start'] < VAL_PHYSICS_END)

# 2. Val Set 2: The "Recency" Test (Summer 2025 — last 3 months)
mask_val_recency = (observed_df['delivery_start'] >= VAL_RECENCY_START)

# 3. Train Set: Everything else
mask_train = ~(mask_val_physics | mask_val_recency)

train_df = observed_df[mask_train]
val_physics_df = observed_df[mask_val_physics]
val_recency_df = observed_df[mask_val_recency]

print(f'🌲 Training rows: {len(train_df):,}')
print(f'❄️ Val 1 (Physics — Autumn/Winter 2024): {len(val_physics_df):,}')
print(f'🔥 Val 2 (Recency — Summer 2025):        {len(val_recency_df):,}')

🌲 Training rows: 101,792
❄️ Val 1 (Physics — Autumn/Winter 2024): 17,568
🔥 Val 2 (Recency — Summer 2025):        13,248


In [42]:
drop_cols = set(['id', 'target', 'market', 'delivery_start', 'delivery_end', 'is_test'])
feat_cols = sorted([c for c in df.columns if c not in drop_cols])
cat_idx = [feat_cols.index('market_id')] if 'market_id' in feat_cols else []

X_train = train_df[feat_cols]
y_train_real = train_df['target'].values
y_train = np.arcsinh(train_df['target'].values)

X_val_physics = val_physics_df[feat_cols]
y_val_physics_real = val_physics_df['target'].values
y_val_physics = np.arcsinh(val_physics_df['target'].values)

X_val_recency = val_recency_df[feat_cols]
y_val_recency_real = val_recency_df['target'].values
y_val_recency = np.arcsinh(val_recency_df['target'].values)

X_all = observed_df[feat_cols]
y_all_real = observed_df['target'].values
y_all = np.arcsinh(observed_df['target'].values)

X_test = test_df[feat_cols]

## PREPARATION & M0 NUCLEAR DROP

In [60]:
import os, json, time
import numpy as np, pandas as pd
import optuna, xgboost as xgb, lightgbm as lgb
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import root_mean_squared_error
from scipy.optimize import minimize as sp_minimize

optuna.logging.set_verbosity(optuna.logging.WARNING)
os.makedirs('../models/v8/', exist_ok=True)
os.makedirs('../submissions/', exist_ok=True)

# ── Market masks on TRAINING data ──
mask_tr_m0   = X_train['market_id'] == 0
mask_tr_rest = X_train['market_id'] > 0

# Nuclear Drop: M0 training only from 2024+
nuclear = train_df['delivery_start'] >= '2024-01-01'
X_train_m0   = X_train[mask_tr_m0 & nuclear].drop(columns=['market_id'])
y_train_m0   = y_train[(mask_tr_m0 & nuclear).values]
X_train_rest = X_train[mask_tr_rest]
y_train_rest = y_train[mask_tr_rest.values]

# ── Market masks on PHYSICS VAL data ──
mask_vp_m0   = X_val_physics['market_id'] == 0
mask_vp_rest = X_val_physics['market_id'] > 0
X_vp_m0      = X_val_physics[mask_vp_m0].drop(columns=['market_id'])
y_vp_m0      = y_val_physics[mask_vp_m0.values]
y_vp_real_m0 = y_val_physics_real[mask_vp_m0.values]
X_vp_rest      = X_val_physics[mask_vp_rest]
y_vp_rest      = y_val_physics[mask_vp_rest.values]
y_vp_real_rest = y_val_physics_real[mask_vp_rest.values]

# Feature list without market_id (for M0 models)
feat_m0 = [c for c in feat_cols if c != 'market_id']

print(f"M0  Train (nuclear): {len(X_train_m0):,}")
print(f"Rest Train:          {len(X_train_rest):,}")
print(f"M0  Val Physics:     {len(X_vp_m0):,}")
print(f"Rest Val Physics:    {len(X_vp_rest):,}")

M0  Train (nuclear): 9,478
Rest Train:          83,555
M0  Val Physics:     2,928
Rest Val Physics:    14,640


## BLOCK A — LightGBM Tuning

In [28]:
t0 = time.time()
print("="*60); print("BLOCK A: LightGBM"); print("="*60)

def _lgb_obj(trial, Xtr, ytr, Xvl, yvl, yvl_real, cat_idx=None):
    p = {'objective':'huber','metric':'rmse','verbosity':-1,'seed':SEED,'n_jobs':-1,
         'learning_rate': trial.suggest_float('lr', 1e-3, 0.1, log=True),
         'num_leaves':    trial.suggest_int('nl', 31, 512),
         'alpha':         trial.suggest_float('alpha', 1.0, 3.0)}
    ds_t = lgb.Dataset(Xtr, ytr, categorical_feature=cat_idx or 'auto', free_raw_data=False)
    ds_v = lgb.Dataset(Xvl, yvl, reference=ds_t, free_raw_data=False)
    m = lgb.train(p, ds_t, 2000, valid_sets=[ds_v],
                  callbacks=[lgb.early_stopping(50, verbose=False)])
    trial.set_user_attr('bi', m.best_iteration)
    return root_mean_squared_error(yvl_real, np.sinh(m.predict(Xvl)))

print("  LGB M0 …")
s_lgb_m0 = optuna.create_study(direction='minimize')
s_lgb_m0.optimize(lambda t: _lgb_obj(t, X_train_m0, y_train_m0,
                                       X_vp_m0, y_vp_m0, y_vp_real_m0), n_trials=25)
print(f"  Best M0  RMSE={s_lgb_m0.best_value:.4f}")

print("  LGB Rest …")
cat_idx_rest = [list(X_train_rest.columns).index('market_id')]
s_lgb_rest = optuna.create_study(direction='minimize')
s_lgb_rest.optimize(lambda t: _lgb_obj(t, X_train_rest, y_train_rest,
                                         X_vp_rest, y_vp_rest, y_vp_real_rest,
                                         cat_idx_rest), n_trials=25)
print(f"  Best Rest RMSE={s_lgb_rest.best_value:.4f}")

# Train final val models with best params
def _lgb_train(best, Xtr, ytr, Xvl, yvl, bi, cat_idx=None):
    p = {'objective':'huber','metric':'rmse','verbosity':-1,'seed':SEED,'n_jobs':-1,
         'learning_rate':best['lr'],'num_leaves':best['nl'],'alpha':best['alpha']}
    ds_t = lgb.Dataset(Xtr, ytr, categorical_feature=cat_idx or 'auto')
    ds_v = lgb.Dataset(Xvl, yvl, reference=ds_t)
    return lgb.train(p, ds_t, bi, valid_sets=[ds_v],
                     callbacks=[lgb.early_stopping(50, verbose=False)])

bi_lgb_m0   = s_lgb_m0.best_trial.user_attrs['bi']
bi_lgb_rest = s_lgb_rest.best_trial.user_attrs['bi']

val_lgb_m0   = _lgb_train(s_lgb_m0.best_params,   X_train_m0, y_train_m0,
                           X_vp_m0, y_vp_m0, bi_lgb_m0)
val_lgb_rest = _lgb_train(s_lgb_rest.best_params, X_train_rest, y_train_rest,
                           X_vp_rest, y_vp_rest, bi_lgb_rest, cat_idx_rest)

val_lgb_m0.save_model('../models/v8/lgb_m0_val.txt')
val_lgb_rest.save_model('../models/v8/lgb_rest_val.txt')

json.dump({'m0':s_lgb_m0.best_params,'m0_bi':bi_lgb_m0,
           'rest':s_lgb_rest.best_params,'rest_bi':bi_lgb_rest},
          open('../models/v8/lgb_best_params.json','w'))
print(f"  LGB done in {time.time()-t0:.0f}s")

BLOCK A: LightGBM
  LGB M0 …
  Best M0  RMSE=80.1025
  LGB Rest …
  Best Rest RMSE=39.2627
  LGB done in 2122s


## BLOCK B — XGBoost Tuning

In [44]:
print("="*60); print("BLOCK B: XGBoost"); print("="*60)

def _xgb_obj(trial, Xtr, ytr, Xvl, yvl, yvl_real):
    # Fix: Changed 'reg:huber' to 'reg:pseudohubererror'
    p = {
        'tree_method': 'hist', 
        'objective': 'reg:pseudohubererror', # Corrected name
        'huber_slope': trial.suggest_float('alpha', 1.0, 3.0), # 'alpha' in LGB is 'huber_slope' in XGB
        'eval_metric': 'rmse',
        'seed': SEED, 
        'n_jobs': -1,
        'learning_rate': trial.suggest_float('lr', 1e-3, 0.1, log=True),
        'max_depth': trial.suggest_int('md', 5, 12)
    }
    dt = xgb.DMatrix(Xtr, label=ytr); dv = xgb.DMatrix(Xvl, label=yvl)
    m = xgb.train(p, dt, 2000, evals=[(dv,'v')],
                  early_stopping_rounds=50, verbose_eval=False)
    trial.set_user_attr('bi', m.best_iteration)
    preds = np.clip(m.predict(dv), -20, 20)  # prevent sinh overflow
    return root_mean_squared_error(yvl_real, np.sinh(preds))

print("  XGB M0 …")
s_xgb_m0 = optuna.create_study(direction='minimize')
s_xgb_m0.optimize(lambda t: _xgb_obj(t, X_train_m0, y_train_m0,
                                       X_vp_m0, y_vp_m0, y_vp_real_m0), n_trials=25)
print(f"  Best M0  RMSE={s_xgb_m0.best_value:.4f}")

print("  XGB Rest …")
s_xgb_rest = optuna.create_study(direction='minimize')
s_xgb_rest.optimize(lambda t: _xgb_obj(t, X_train_rest, y_train_rest,
                                         X_vp_rest, y_vp_rest, y_vp_real_rest), n_trials=25)
print(f"  Best Rest RMSE={s_xgb_rest.best_value:.4f}")

# --- Retrain Best Models for Scipy Weights ---
def _xgb_train(best, Xtr, ytr, Xvl, yvl, bi):
    p = {
        'tree_method': 'hist', 
        'objective': 'reg:pseudohubererror', # Corrected name
        'huber_slope': best['alpha'],
        'eval_metric': 'rmse',
        'seed': SEED, 
        'n_jobs': -1, 
        'learning_rate': best['lr'], 
        'max_depth': best['md']
    }
    dt = xgb.DMatrix(Xtr, label=ytr); dv = xgb.DMatrix(Xvl, label=yvl)
    return xgb.train(p, dt, bi, evals=[(dv,'v')],
                     early_stopping_rounds=50, verbose_eval=False)

bi_xgb_m0   = s_xgb_m0.best_trial.user_attrs['bi']
bi_xgb_rest = s_xgb_rest.best_trial.user_attrs['bi']

val_xgb_m0   = _xgb_train(s_xgb_m0.best_params,   X_train_m0, y_train_m0,
                            X_vp_m0, y_vp_m0, bi_xgb_m0)
val_xgb_rest = _xgb_train(s_xgb_rest.best_params, X_train_rest, y_train_rest,
                            X_vp_rest, y_vp_rest, bi_xgb_rest)

# Save validation models and params
val_xgb_m0.save_model('../models/v8/xgb_m0_val.json')
val_xgb_rest.save_model('../models/v8/xgb_rest_val.json')

json.dump({'m0':s_xgb_m0.best_params,'m0_bi':bi_xgb_m0,
           'rest':s_xgb_rest.best_params,'rest_bi':bi_xgb_rest},
          open('../models/v8/xgb_best_params.json','w'))
print(f"  XGB done.")

BLOCK B: XGBoost
  XGB M0 …
  Best M0  RMSE=77.1926
  XGB Rest …
  Best Rest RMSE=38.4633
  XGB done.


## BLOCK C — CatBoost Tuning

In [45]:
t0 = time.time()
print("="*60); print("BLOCK C: CatBoost"); print("="*60)

def _cat_obj(trial, Xtr, ytr, Xvl, yvl, yvl_real, cat_feat=None):
    p = {'loss_function':'Huber:delta=1.5','task_type':'CPU','eval_metric':'RMSE',
         'random_seed':SEED,'verbose':False,'iterations':1000,'early_stopping_rounds':50,
         'learning_rate': trial.suggest_float('lr', 1e-3, 0.1, log=True),
         'depth':         trial.suggest_int('depth', 4, 10)}
    m = CatBoostRegressor(**p)
    if cat_feat:
        m.fit(Pool(Xtr, ytr, cat_features=cat_feat),
              eval_set=Pool(Xvl, yvl, cat_features=cat_feat), use_best_model=True)
    else:
        m.fit(Xtr, ytr, eval_set=(Xvl, yvl), use_best_model=True)
    trial.set_user_attr('bi', m.get_best_iteration())
    preds = np.clip(m.predict(Xvl), -20, 20)  # prevent sinh overflow
    return root_mean_squared_error(yvl_real, np.sinh(preds))

print("  CAT M0 …")
s_cat_m0 = optuna.create_study(direction='minimize')
s_cat_m0.optimize(lambda t: _cat_obj(t, X_train_m0, y_train_m0,
                                      X_vp_m0, y_vp_m0, y_vp_real_m0), n_trials=15)
print(f"  Best M0  RMSE={s_cat_m0.best_value:.4f}")

print("  CAT Rest …")
cat_feat_rest = [list(X_train_rest.columns).index('market_id')]
s_cat_rest = optuna.create_study(direction='minimize')
s_cat_rest.optimize(lambda t: _cat_obj(t, X_train_rest, y_train_rest,
                                        X_vp_rest, y_vp_rest, y_vp_real_rest,
                                        cat_feat_rest), n_trials=15)
print(f"  Best Rest RMSE={s_cat_rest.best_value:.4f}")

# Train final val models
def _cat_train(best, Xtr, ytr, Xvl, yvl, bi, cat_feat=None):
    p = {'loss_function':'Huber:delta=1.5','task_type':'CPU','eval_metric':'RMSE',
         'random_seed':SEED,'verbose':False,'iterations':bi,
         'learning_rate':best['lr'],'depth':best['depth']}
    m = CatBoostRegressor(**p)
    if cat_feat:
        m.fit(Pool(Xtr, ytr, cat_features=cat_feat),
              eval_set=Pool(Xvl, yvl, cat_features=cat_feat), use_best_model=True)
    else:
        m.fit(Xtr, ytr, eval_set=(Xvl, yvl), use_best_model=True)
    return m

bi_cat_m0   = s_cat_m0.best_trial.user_attrs['bi']
bi_cat_rest = s_cat_rest.best_trial.user_attrs['bi']

val_cat_m0   = _cat_train(s_cat_m0.best_params,   X_train_m0, y_train_m0,
                           X_vp_m0, y_vp_m0, bi_cat_m0)
val_cat_rest = _cat_train(s_cat_rest.best_params, X_train_rest, y_train_rest,
                           X_vp_rest, y_vp_rest, bi_cat_rest, cat_feat_rest)

val_cat_m0.save_model('../models/v8/cat_m0_val.cbm')
val_cat_rest.save_model('../models/v8/cat_rest_val.cbm')

json.dump({'m0':s_cat_m0.best_params,'m0_bi':bi_cat_m0,
           'rest':s_cat_rest.best_params,'rest_bi':bi_cat_rest},
          open('../models/v8/cat_best_params.json','w'))
print(f"  CAT done in {time.time()-t0:.0f}s")

BLOCK C: CatBoost
  CAT M0 …
  Best M0  RMSE=79.1533
  CAT Rest …
  Best Rest RMSE=39.5610
  CAT done in 1945s


## BLOCK D — Scipy Blend-Weight Optimization

In [46]:
print("="*60); print("BLOCK D: SCIPY WEIGHT OPTIMIZATION"); print("="*60)

# Val predictions from 6 validation-trained models
vp_lgb_m0  = np.sinh(np.clip(val_lgb_m0.predict(X_vp_m0), -20, 20))
vp_xgb_m0  = np.sinh(np.clip(val_xgb_m0.predict(xgb.DMatrix(X_vp_m0)), -20, 20))
vp_cat_m0  = np.sinh(np.clip(val_cat_m0.predict(X_vp_m0), -20, 20))

vp_lgb_r   = np.sinh(np.clip(val_lgb_rest.predict(X_vp_rest), -20, 20))
vp_xgb_r   = np.sinh(np.clip(val_xgb_rest.predict(xgb.DMatrix(X_vp_rest)), -20, 20))
vp_cat_r   = np.sinh(np.clip(val_cat_rest.predict(X_vp_rest), -20, 20))

def _blend_rmse(w, p1, p2, p3, y_real):
    return root_mean_squared_error(y_real, w[0]*p1 + w[1]*p2 + w[2]*p3)

cons = {'type':'eq','fun': lambda w: w.sum()-1.0}
bnd  = [(0,1)]*3
w0   = [1/3, 1/3, 1/3]

res_m0   = sp_minimize(_blend_rmse, w0, args=(vp_lgb_m0, vp_xgb_m0, vp_cat_m0, y_vp_real_m0),
                       method='SLSQP', bounds=bnd, constraints=cons)
res_rest = sp_minimize(_blend_rmse, w0, args=(vp_lgb_r,  vp_xgb_r,  vp_cat_r,  y_vp_real_rest),
                       method='SLSQP', bounds=bnd, constraints=cons)

best_w_m0   = res_m0.x
best_w_rest = res_rest.x

print(f"  M0   weights [LGB,XGB,CAT]: {np.round(best_w_m0, 4)}  RMSE={res_m0.fun:.4f}")
print(f"  Rest weights [LGB,XGB,CAT]: {np.round(best_w_rest, 4)}  RMSE={res_rest.fun:.4f}")

json.dump({'m0': best_w_m0.tolist(), 'rest': best_w_rest.tolist()},
          open('../models/v8/ensemble_weights.json','w'))

BLOCK D: SCIPY WEIGHT OPTIMIZATION
  M0   weights [LGB,XGB,CAT]: [0. 1. 0.]  RMSE=77.1912
  Rest weights [LGB,XGB,CAT]: [0.0852 0.9105 0.0042]  RMSE=38.4495


## BLOCK E — Full Retraining on All Observed Data

In [47]:
print("="*60); print("BLOCK E: FULL RETRAINING"); print("="*60)

# Nuclear drop on X_all for M0
mask_all_m0   = X_all['market_id'] == 0
mask_all_rest = X_all['market_id'] > 0
nuc_all       = observed_df['delivery_start'] >= '2024-01-01'

X_all_m0   = X_all[mask_all_m0 & nuc_all].drop(columns=['market_id'])
y_all_m0   = y_all[(mask_all_m0 & nuc_all).values]
X_all_rest = X_all[mask_all_rest]
y_all_rest = y_all[mask_all_rest.values]

print(f"  Full M0 (nuclear): {len(X_all_m0):,}   Rest: {len(X_all_rest):,}")

# ── LGB final ──
p = {'objective':'huber','metric':'rmse','verbosity':-1,'seed':SEED,'n_jobs':-1}
p.update({k.replace('lr','learning_rate').replace('nl','num_leaves'):v
          for k,v in s_lgb_m0.best_params.items()})
# fix key names
_p = {'objective':'huber','metric':'rmse','verbosity':-1,'seed':SEED,'n_jobs':-1,
      'learning_rate':s_lgb_m0.best_params['lr'],
      'num_leaves':s_lgb_m0.best_params['nl'],
      'alpha':s_lgb_m0.best_params['alpha']}
lgb_m0_final = lgb.train(_p, lgb.Dataset(X_all_m0, y_all_m0),
                          num_boost_round=int(bi_lgb_m0*1.1))

_p = {'objective':'huber','metric':'rmse','verbosity':-1,'seed':SEED,'n_jobs':-1,
      'learning_rate':s_lgb_rest.best_params['lr'],
      'num_leaves':s_lgb_rest.best_params['nl'],
      'alpha':s_lgb_rest.best_params['alpha']}
cat_idx_rest = [list(X_all_rest.columns).index('market_id')]
lgb_rest_final = lgb.train(_p, lgb.Dataset(X_all_rest, y_all_rest, categorical_feature=cat_idx_rest),
                            num_boost_round=int(bi_lgb_rest*1.1))

lgb_m0_final.save_model('../models/v8/lgb_m0.txt')
lgb_rest_final.save_model('../models/v8/lgb_rest.txt')

# ── XGB final ──
_p = {'tree_method':'hist','objective':'reg:pseudohubererror','eval_metric':'rmse',
      'seed':SEED,'n_jobs':-1,
      'learning_rate':s_xgb_m0.best_params['lr'],
      'max_depth':s_xgb_m0.best_params['md'],
      'huber_slope':s_xgb_m0.best_params['alpha']}
xgb_m0_final = xgb.train(_p, xgb.DMatrix(X_all_m0, label=y_all_m0),
                           num_boost_round=int(bi_xgb_m0*1.1))

_p = {'tree_method':'hist','objective':'reg:pseudohubererror','eval_metric':'rmse',
      'seed':SEED,'n_jobs':-1,
      'learning_rate':s_xgb_rest.best_params['lr'],
      'max_depth':s_xgb_rest.best_params['md'],
      'huber_slope':s_xgb_rest.best_params['alpha']}
xgb_rest_final = xgb.train(_p, xgb.DMatrix(X_all_rest, label=y_all_rest),
                             num_boost_round=int(bi_xgb_rest*1.1))

xgb_m0_final.save_model('../models/v8/xgb_m0.json')
xgb_rest_final.save_model('../models/v8/xgb_rest.json')

# ── CAT final ──
_p = {'loss_function':'Huber:delta=1.5','task_type':'CPU','random_seed':SEED,'verbose':False,
      'learning_rate':s_cat_m0.best_params['lr'],
      'depth':s_cat_m0.best_params['depth'],
      'iterations':int(bi_cat_m0*1.1)}
cat_m0_final = CatBoostRegressor(**_p)
cat_m0_final.fit(X_all_m0, y_all_m0)

_p = {'loss_function':'Huber:delta=1.5','task_type':'CPU','random_seed':SEED,'verbose':False,
      'learning_rate':s_cat_rest.best_params['lr'],
      'depth':s_cat_rest.best_params['depth'],
      'iterations':int(bi_cat_rest*1.1)}
cat_rest_final = CatBoostRegressor(**_p)
cat_rest_final.fit(Pool(X_all_rest, y_all_rest,
                        cat_features=[list(X_all_rest.columns).index('market_id')]))

cat_m0_final.save_model('../models/v8/cat_m0.cbm')
cat_rest_final.save_model('../models/v8/cat_rest.cbm')

print("  All 6 final models saved.")

BLOCK E: FULL RETRAINING
  Full M0 (nuclear): 14,614   Rest: 109,235
  All 6 final models saved.


## BLOCK F — Ensemble Inference & Submission

In [61]:
print("="*60); print("BLOCK F: ENSEMBLE INFERENCE"); print("="*60)

mask_te_m0   = X_test['market_id'] == 0
mask_te_rest = X_test['market_id'] > 0

Xt_m0   = X_test[mask_te_m0].drop(columns=['market_id'])
Xt_rest = X_test[mask_te_rest]

# Predictions (arcsinh → sinh)
lg_m0 = np.sinh(np.clip(lgb_m0_final.predict(Xt_m0), -20, 20))
lg_r  = np.sinh(np.clip(lgb_rest_final.predict(Xt_rest), -20, 20))
xg_m0 = np.sinh(np.clip(xgb_m0_final.predict(xgb.DMatrix(Xt_m0)), -20, 20))
xg_r  = np.sinh(np.clip(xgb_rest_final.predict(xgb.DMatrix(Xt_rest)), -20, 20))
ct_m0 = np.sinh(np.clip(cat_m0_final.predict(Xt_m0), -20, 20))
ct_r  = np.sinh(np.clip(cat_rest_final.predict(Xt_rest), -20, 20))

# Blend using Scipy-optimized weights
final_preds = np.empty(len(X_test))
final_preds[mask_te_m0.values]   = best_w_m0[0]*lg_m0 + best_w_m0[1]*xg_m0 + best_w_m0[2]*ct_m0
final_preds[mask_te_rest.values] = best_w_rest[0]*lg_r + best_w_rest[1]*xg_r + best_w_rest[2]*ct_r

# Build submission
pred_df = pd.DataFrame({"id": test_df["id"].values, "target": final_preds})
submission = sample_sub[["id"]].merge(pred_df, on="id", how="left")

assert len(submission) == len(sample_sub)
assert submission["target"].isna().sum() == 0
assert (submission["id"] == sample_sub["id"]).all()

path = "../submissions/submission_v8_ensemble.csv"
submission.to_csv(path, index=False)

print(f"✅ Saved: {path}")
print(f"   Mean:   {submission['target'].mean():.4f}")
print(f"   Std:    {submission['target'].std():.4f}")
print(f"   Min:    {submission['target'].min():.4f}")
print(f"   Max:    {submission['target'].max():.4f}")
print(f"   Median: {submission['target'].median():.4f}")

BLOCK F: ENSEMBLE INFERENCE
✅ Saved: ../submissions/submission_v8_ensemble.csv
   Mean:   31.8885
   Std:    30.0919
   Min:    -122.7772
   Max:    359.4650
   Median: 27.3252
